In [22]:
'''
多线程版本爬取漫画，突破ip封锁
'''
import os
import requests
import random
from selenium import webdriver
# 线程池
from multiprocessing.dummy import Pool as ThreadPool
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities  
from selenium.webdriver.common.proxy import Proxy
from selenium.webdriver.common.proxy import ProxyType  

'''
从一个被良好维护的user-agent列表里
随机筛选合适的user-agent 
防止封锁
'''
#一个不容易被封锁的user-agent列表
agents = ['Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0;',
              'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv,2.0.1) Gecko/20100101 Firefox/4.0.1',
              'Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11',
              'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11',
              'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)']
'''
通过爬取可用的免费代理，
进行测试可用度之后
筛选获得的代理
'''
proxies = [
    '89.36.215.72:1189',
    '94.177.203.123:1189',
    '110.73.11.227:8123',
    '180.183.176.189:8080',
    '109.62.247.81:8080',
]
_proxy = random.choice(proxies)

proxy = Proxy(
    {
        'proxyType': ProxyType.MANUAL,
        'httpProxy': _proxy  # 代理ip和端口
    }
)

desired_capabilities = DesiredCapabilities.PHANTOMJS.copy()
# 将代理ip加入
proxy.add_to_capabilities(desired_capabilities)
desired_capabilities["phantomjs.page.settings.userAgent"] = (random.choice(agents))  

class Item(object):
    '''
    保存漫画的章节名和url地址
    '''
    url = None
    name = None

class Op(object):
    def __init__(self):
        '''
        初始化
        '''
        self.index_url = self.get_TOF('http://manhua.sfacg.com/mh/YiWang/')
        self.items = self.get_pic(self.index_url)
        # 开启多线程，线程数为10
        pool = ThreadPool(10)
        pool.map(self.SavePic, self.items)
        pool.close()
        pool.join()
    def mkdir(self, path):
        '''
        防止目录存在
        '''
        if not os.path.exists(path):
            try:
                os.mkdir(path)
            # 调试错误    
            except Exception as err:
                print(path + '---')
                print(err)
                
    def SavePic(self, item):
        '''
        将抓取到的图片保存到本地
        '''
        url = item.url
        filename = item.name
        content = requests.get(url).content
        # 二进制格式写入
        with open(filename, 'wb') as f:
            f.write(content)
        print('当前文件\t%s\t下载完毕' % filename)    
    def get_TOF(self, index_url):
        '''
        获取每一章节的url并与漫画名一起返回字典格式数据
        '''
        url_list = []
        # 模拟浏览器打开网页
        browser = webdriver.PhantomJS(executable_path="phantomjs",
desired_capabilities=desired_capabilities)
        browser.get(index_url)
        browser.implicitly_wait(3)
        
        # 找到漫画标题 并创建目录
        title = browser.title.split(',')[0]
        print(title + '11==')
        self.mkdir(title)
        
        # 找到漫画章节及其他番外，短片之类
        comics_lists = browser.find_elements_by_class_name('comic_Serial_list')
        # 查找正文等
        for part in comics_lists:
            # 找到包裹链接的links
            links = part.find_elements_by_tag_name('a')
            # 再找到每个单独章节的链接
            for link in links:
                url_list.append(link.get_attribute('href'))
         # 关闭连接
        browser.quit()
        # 字典对象存储
        Comics = dict(name=title, urls=url_list)
        return Comics
    def get_pic(self, Comics):
        '''
        打开每个章节并找到漫画图片的url然后保存到列表并返回
        '''
        # 用于保存item的列表
        comics_urls = []
        # 从dict中拿取漫画名和章节链接
        comic_list = Comics['urls']
        basedir = Comics['name']
        
        # 构建浏览器对象
        browser = webdriver.PhantomJS(executable_path="phantomjs",
desired_capabilities=desired_capabilities)
        for url in comic_list:
            browser.get(url)
            browser.implicitly_wait(3)
            
            # 创建章节目录
            dirname = basedir + '/' + browser.title.split('-')[1]
            self.mkdir(dirname)
            # 找到该漫画的页数
            pageNum = len(browser.find_elements_by_tag_name('option'))
            # 找到下一页的事件触发
            nextpage = browser.find_element_by_xpath("//*[@id='AD_j1']/div/a[4]")
            # 找到图片地址并触发下一页操作
            for i in range(pageNum):
                # item对象，需要的数据存储
                item = Item()
                item.url = browser.find_element_by_id('curPic').get_attribute('src')
                item.name = dirname +'/'+ str(i) + '.png'
                comics_urls.append(item)
                # 下一页
                nextpage.click()
            # 退出连接
            browser.quit()
        return comics_urls
if __name__ == '__main__':
    Go = Op()

URLError: <urlopen error [WinError 10061] 由于目标计算机积极拒绝，无法连接。>